In [1]:
import pandas as pd													# data manipulation
import matplotlib.pyplot as plt												# graphs
import seaborn as sns													# graphs
import numpy as np													# graphs
from scipy.stats import norm												# statistics
from sklearn.preprocessing import StandardScaler									# statistics
from scipy import stats													# statistics
from math import *													# advance calculation	
from matplotlib import cm												# graphs
import sys														# system functions
import warnings														# indicating warnings
warnings.filterwarnings('ignore')											# ignore by default
from matplotlib.ticker import LinearLocator										# graphs
pd.set_option('display.max_columns', None)										# show all columns
pd.set_option('display.max_rows', 100)											# show all rows
np.set_printoptions(threshold=sys.maxsize)										# show arrays full
pd.set_option("display.max_colwidth", -1)										# show a complete text in a cell
pd.set_option('display.precision',5)											# precision decimals pd
np.set_printoptions(precision=3)											# precision decimals np
from xlsxwriter.workbook import Workbook										# CSV writing
import csv														# CSV writing	
import xgboost as xgb													# prediction
import pickle														# prediction	
import sklearn														# mutual information
import statsmodels.api as sm												# coefficient
from sklearn.ensemble import RandomForestClassifier									# random forest
from sklearn import linear_model											# linear regression
import pymysql														# MySql connection
import psycopg2														# PostgreSQL database adapter
import datetime														# time conversions
import random														# randomchoice
import math as math													# maths
import statistics													# statistics
import glob														# paths and files management
import matplotlib.image as mpimg											# matplot image
from functools import reduce												# reduce

In [31]:
# Reading 2 tables from the AWS
conn = psycopg2.connect(
    host="da-assignment-db.cgnpeyd28dyn.eu-west-1.rds.amazonaws.com",
    port="5432",
    database="postgres",
    user="da_user",
    password="BNjv!9Met3")

Query="SELECT * FROM assignment.page_views_da;"
cursor=conn.cursor()
cursor.execute(Query)
rows=cursor.fetchall()
column_names = [i[0] for i in cursor.description]
fp = open('assignment.page_views_da.csv', 'w')
myFile = csv.writer(fp, lineterminator = '\n')
myFile.writerow(column_names)   
myFile.writerows(rows)
fp.close()

Query="SELECT * FROM assignment.sessions;"
cursor=conn.cursor()
cursor.execute(Query)
rows=cursor.fetchall()
column_names = [i[0] for i in cursor.description]
fp = open('assignment.sessions.csv', 'w', encoding="utf_8")
myFile = csv.writer(fp, lineterminator = '\n')
myFile.writerow(column_names)   
myFile.writerows(rows)
fp.close()

# Merge, sort, deal with white spaces
dfp = pd.read_csv('assignment.page_views_da.csv')
dfs = pd.read_csv('assignment.sessions.csv', encoding='utf_8')
dfs.rename(columns={'id':'session_id'}, inplace=True)
dfp['time_of_day'] = pd.to_timedelta(dfp['time_of_day'])
df = dfp.merge(dfs, on="session_id")
df = df.sort_values(['user_fingerprint', 'day_number','time_of_day'], ascending=[True, True, True])
df = df.reset_index(drop=True)
for column in df.columns.drop(['day_number','time_of_day','ua_is_mobile','ua_is_pc','ua_is_tablet']):
    df[column] = df[column].str.strip()
df.columns = ['Day', 'Time', 'SessionID', 'UserID','PageType','ConfirmationID','Mobile','PC','Tablet','OS','City','Country']
dftarget2 = df.loc[df['ConfirmationID'].notnull()]
for index in dftarget2.index:
    if dftarget2.duplicated(subset=['ConfirmationID'])[index] == True:
        df.at[index, 'ConfirmationID'] = np.nan

In [62]:
# 1a answer (Via SQL)
Query="SELECT CONCAT(ROUND(COUNT(DISTINCT a.confirmation_id)::decimal/COUNT(DISTINCT a.session_id)*100,2),'%') AS ConversionRate \
    FROM assignment.page_views_da a \
    LEFT JOIN assignment.sessions b ON a.session_id=b.id \
    WHERE b.ua_is_mobile =True;"

cursor=conn.cursor()
cursor.execute(Query)
print('1a answer is:',cursor.fetchall())

1a answer is: [('3.19%',)]


In [75]:
# 1b answer (Via Python)
def count1(df):
    i=0
    a = [''] * 300000
    for userid in df['user_fingerprint'].unique():
        dftarget = df.loc[df['user_fingerprint'] == userid]
        if dftarget['session_id'].nunique() > 1: 
            for item in range(1, len(dftarget['session_id'].unique())): 
                if dftarget.loc[dftarget['session_id'] == dftarget['session_id'].unique()[item]]['page_type'].head(1).sum() == 'product':
                    for session in dftarget['session_id'].unique()[item:]:
                        a[i] = session
                        i=i+1
    return a

session_id = count1(df)
dftarget = df.loc[df['session_id'].isin(session_id)]
print('1b answer is:',round(dftarget.shape[0]/dftarget['session_id'].nunique(),2))

1b answer is: 2.51


In [97]:
# Funnel analysis in order
print(df['SessionID'].nunique())
print(df.loc[(df['PageType'] == 'product') | (df['PageType'] == 'location')]['SessionID'].nunique())
print(df.loc[df['PageType'] == 'bookingDetails']['SessionID'].nunique())
print(df.loc[df['PageType'] == 'personalDetails']['SessionID'].nunique())
print(df.loc[df['PageType'] == 'payment']['SessionID'].nunique())
print(df['ConfirmationID'].nunique())

365742
292040
89426
20680
12209
10506


In [4]:
# List of returning sessions
def solve(df):
    i=0
    a = [''] * 100000
    for user in df['UserID'].unique():
        dftarget = df.loc[df['UserID'] == user]
        if dftarget['SessionID'].nunique() > 1:
             for session in dftarget['SessionID'].unique()[1:]:
                a[i] = session
                i=i+1
    return a
a = solve(df)

# Day, SessionID, Confirmation
# ReturningSession, BounceRate(Later), SessionDuration, EventsPerSession
# LandingPage, Device, OS, City, Country
def result(df):
    dfresult = pd.DataFrame(data={'Day':[''],'SessionID':[''],'Confirmation':[''],'Returning':[''],'SessionDuration':[''],
    'Event':[''],'LandingPage':[''],'Device':[''], 'OS':[''],'City':[''],'Country':['']},
    columns=['Day','SessionID','Confirmation','Returning','SessionDuration','Event','LandingPage','Device','OS','City','Country'])
    index = 0
    for session in df['SessionID'].unique():
        dftarget = df.loc[df['SessionID'] == session]
        dfresult.at[index, 'Day'] = dftarget['Day'].head(1).sum()
        dfresult.at[index, 'SessionID'] = session
        dfresult.at[index, 'Confirmation'] = dftarget['ConfirmationID'].notnull().sum()
        if session in a:
            dfresult.at[index, 'Returning'] = 1
        else:
            dfresult.at[index, 'Returning'] = 0
        dfresult.at[index, 'SessionDuration'] = sessionduration = round((dftarget['Time'].tail(1).sum() - dftarget['Time'].head(1).sum()) / np.timedelta64(1, 's'),0)
        if sessionduration == 0:
            dfresult.at[index, 'SessionDuration'] = np.nan
        dfresult.at[index, 'Event'] = dftarget.shape[0]
        dfresult.at[index, 'LandingPage'] = dftarget['PageType'].head(1).sum()
        if dftarget['Mobile'].head(1).sum() == 1:
            dfresult.at[index, 'Device'] = 'Mobile'
        elif dftarget['PC'].head(1).sum() == 1:
            dfresult.at[index, 'Device'] = 'PC'
        else:
            dfresult.at[index, 'Device'] = 'OS'
        dfresult.at[index, 'OS'] = dftarget['OS'].head(1).sum()
        dfresult.at[index, 'City'] = city = dftarget['City'].head(1).sum()
        if city == 0:
            dfresult.at[index, 'City'] = np.nan
        dfresult.at[index, 'Country'] = dftarget['Country'].head(1).sum()
        index=index + 1
    return dfresult
dfresult = result(df)
dfresult.to_csv('dfresult.csv', index=None)

In [6]:
df = pd.read_csv('dfresult.csv')
df.at[df.loc[df['Device'] == 'OS'].index,'Device'] = 'Tablet'

In [87]:
# Returning Sessions

round(df.loc[df['Returning'] == 1].shape[0]/df.shape[0]*100,2)

25.57

In [84]:
# Tiqets bounce rate:

round(df.loc[((df['SessionDuration'] > 60) & (df['Event'] == 2)) | (df['Event'] == 1) ].shape[0]/df.shape[0]*100,2)

64.14

In [79]:
# ConfirmationPerSession for bounces 

dftarget1 = df.loc[((df['SessionDuration'] > 60) & (df['Event'] == 2)) | (df['Event'] == 1) ]
round(dftarget1['Confirmation'].mean()*100,2)


0.85

In [81]:
# ConfirmationPerSession for non-bounces 

dftarget2 = df.loc[df['Event'] != 1 ]
round(dftarget2['Confirmation'].mean()*100,2)


5.74

In [101]:
# Average Session Duration (01:57)

round(df['SessionDuration'].sum()/df.shape[0]/60,2)

1.96

In [96]:
# Events Per Session

round(df['Event'].mean(),2)

2.15

In [94]:
df.head(10)

,Day,SessionID,Confirmation,Returning,SessionDuration,Event,LandingPage,Device,OS,City,Country
0,12,9f47e9af13b1a53294e66906f17a4709,0,0,NaN,1,location,Mobile,iOS,Gignac-la-Nerthe,FR
1,5,1921684f7b0a57a1a62618d750cd0de1,0,0,NaN,1,location,Mobile,iOS,Vienna,AT
2,13,dad3041cd185baa10783f2abacbb6199,0,0,395.0,4,product,PC,Mac OS X,Annen,NL
3,11,d4c30e35e9fc50306853a94d4cbe64c1,0,0,NaN,1,product,PC,Windows,Beitostolen,NO
4,12,394a0b29dd122e17750199419cd69ddc,0,0,46.0,2,product,Mobile,iOS,Vancouver,CA
5,6,ccf9be4089b5acff3b539cfc40a6eeb3,0,0,1230.0,4,product,PC,Mac OS X,Chicago,US
6,8,2e3d11bdb53e1b9ad43ac52302cd586b,0,1,9.0,3,product,PC,Mac OS X,Chicago,US
7,10,83ba21e31881ec5ec5bfbd879c4456dc,0,1,NaN,1,product,PC,Mac OS X,Chicago,US
8,10,5da8a9576dc85f7879b9ed8aa5d9c586,0,1,NaN,1,product,PC,Mac OS X,Chicago,US
9,8,2df5a5b0eeb00b241b86c7ccc3de2672,0,0,NaN,1,location,Mobile,Android,Vienna,AT
